In [4]:
import os
import pandas as pd
from lxml import html
import requests
import re
import sys
from datetime import datetime

from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import time

In [5]:
cur_wd = str(os.getcwd())
cur_wd = os.path.join("\\".join(cur_wd.split('\\')[0:-1]),'data')
print(cur_wd)                                 

C:\Users\jjanko\Desktop\wei_mutual_fund_project\data


In [6]:
def find(s, ch):
    return [i for i, ltr in enumerate(s) if ltr == ch]


In [7]:
# cur_wd
parent_path = os.path.dirname(os.getcwd())
mypath = os.path.join((parent_path)+"/"+ "getting_txt_codes"+"/"+"N-30D")
mypath

'C:\\Users\\jjanko\\Desktop\\wei_mutual_fund_project/getting_txt_codes/N-30D'

Find all ciks in the directory

In [8]:
from os.path import isfile, join
from os import listdir
cik_list = [f for f in listdir(mypath) if isfile(join(mypath, f))]
cik_list
CIK = []

for i in cik_list:
    
    try:
        files = pd.read_excel(os.path.join(mypath,i), header = None)
    except: 
        continue


    for index, row in files.iterrows():
        mystr = str(row[0])
        cik_add = re.findall(r'\((.*?)\)',mystr)
        if len(cik_add) == 0:
            print(row[0])
        for i in cik_add:
            if 'CIK ' in i:
                if int(float(i.replace('CIK ', ""))) not in CIK:
                    CIK.append(str(int(float(i.replace('CIK ', "")))))
            
print(CIK)

['5007', '1029446', '726352', '1220781', '797461', '795422', '729218', '754510', '859766', '719451', '722574', '35341', '35315', '311884', '718891', '878662', '819118', '24238', '744822', '35373', '28540', '320351', '35330', '1061130', '880195', '315700', '823535', '275309', '354046', '23355', '740858', '355316', '929964', '763859', '43498', '1003457', '932683', '856671', '45291', '26262', '759866', '22370', '791271', '759828', '888154', '868549', '863328', '792858', '811921', '857769', '785303', '743861', '925683', '315554', '750741', '899581', '855886', '357238', '45288', '807639', '314721', '852954', '1176199', '1189740', '894088', '894089', '1016937', '912035', '916118', '814078', '1122692', '1023771', '916687', '919808', '918686', '904112', '878929', '921187', '912734', '916618', '741375', '809584', '1072377', '837529', '314367', '1116473', '1081658', '1096006', '1048710', '1125801', '314366', '932843', '932842', '745992', '752372', '1031594', '1109076', '726291', '817841', '10078

In [9]:
weblink = {}

# Comment cout CIK_LIST when in production

CIK_LIST = CIK

error = []
txts = []
for CIK in CIK_LIST:
    weblink[CIK] = []
    direc = os.path.join(cur_wd, CIK, 'nd-30')
    if not os.path.exists(direc):
        os.makedirs(direc)
        
        req = Request(r"https://www.sec.gov/Archives/edgar/data/" + str(CIK))
        html_page = urlopen(req, timeout=10)

        soup = BeautifulSoup(html_page, "lxml")

        for link in soup.findAll('a'):
            archive = link.get('href')
            if str('/Archives/edgar/data/') + CIK in archive:
                try:
                    req_archive = Request(r"https://www.sec.gov" + archive)

                    html_page_archive = urlopen(req_archive, timeout=10)
                    soup_archive = BeautifulSoup(html_page_archive, "lxml")

                    for link_archive in soup_archive.findAll('a'):
                        txt_name = link_archive.get('href')
                        if ".txt" in txt_name:
                            ixs = find(txt_name, r'/')
                            file_name = txt_name[ixs[-1]+1:]
                            non_dig = re.findall(r'\D+', file_name)
                            if len(non_dig) == 3:
                                print(r"https://www.sec.gov" + txt_name)
                                try: 
                                    r = Request(r"https://www.sec.gov" + txt_name)
                                    read = urlopen(r, timeout=10)
                                    r = read.readlines()
                                    found = False
                                    #check to see if n-q file
                                    for i in r:
                                        string_row = i.decode('utf-8').lower()
                                        if 'type' in string_row and 'n-30d'in string_row:
                                            found = True
                                except IncompleteRead:
                                    found = False

                                if found:
                                    print(r"https://www.sec.gov" + txt_name)
                                    weblink[CIK].append(r"https://www.sec.gov" + txt_name)
                                    ixs = find(txt_name, r'/')
                                    file_name = txt_name[ixs[-1]+1:]
                                    f = open(os.path.join(direc, file_name), mode = 'w')
                                    for row in r:
                                        clean_row = row.decode('utf-8')
                                        if clean_row != '':
                                            f.write(clean_row)
                                    f.close()
                except:
                    error.append(r"https://www.sec.gov" + archive)

In [11]:
error

[]